# Importing libraries

Importing global and local libraries used explicitly in this notebook. Keep in mind that other libraries are required to run the local library's functions.

In [1]:
# Global libraries
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wg
import sys
# Add src folder to sys path to import local libraries
sys.path.append("../src")
# Importing functions from local libraries located in "src" folder
from support import readFileDialog, parse_data_file
from visualizations import showFrame, keypointsDATAtoFrame

Configure matplotlib plotting to allow for zoom features.

In [2]:
# Try command for jupyter lab, if it doesn't work try for jupyter notebook
try:
    # jupyter lab command
    %matplotlib widget
except:
    # jupyter notebook command
    %matplotlib notebook
# If using zoom features aren't required uncomment next line
# %matplotlib inline

# Open Openpose file

Opening openpose files. The example file is located at "/examples/rowing_ref_op.data".

In [3]:
# file dialog to open the openpose file location
file_path_op = readFileDialog("Open Data file (OP)", "data")
print(file_path_op) # print openpose file location

C:/Users/victo/Documents/Motion_analysis/ema_motion_analysis/examples/rowing_ref_op.data


In [26]:
# store openpose data in a dictionary (data) with keys stored in the var_names list
data, var_names = parse_data_file(file_path_op)
kp_op = data["keypoints"] # store keypoints in a separate variable
md_op = data["metadata"] # store keypoints in a separate variable
ang_op = data["angles"] # store angles in a separate variable
fps_op = md_op["fps"] # store fps in a separate variable
ang_names = md_op['angles_names'] # store angles names in a separate variable
n_frames_op = md_op['n_frames'] # store number of frames in a separate variable
# generate time vector based on the length and fps of the data
t_op = np.linspace(0, len(ang_op)*(1/fps_op), len(ang_op))

# Open Vicon file

Opening vicon files. The example file is located at "/examples/rowing_ref_vicon.data".

In [27]:
# file dialog to open the vicon file location
file_path_vicon = readFileDialog("Open Data file (Vicon)", "data")
print(file_path_vicon) # print vicon file location

C:/Users/victo/Documents/Motion_analysis/Vicon/Celio_Vicon/Voga24_5min_Celio_f_5seg.data


In [28]:
# store vicon data in a dictionary (data) with keys stored in the var_names list
data, var_names = parse_data_file(file_path_vicon)
kp_vicon = data["keypoints"] # store keypoints in a separate variable
md_vicon = data["metadata"] # store keypoints in a separate variable
ang_vicon = data["angles"] # store angles in a separate variable
fps_vicon = md_vicon["fps"] # store fps in a separate variable
# assert vicon and openpose have the same angles order
assert ang_names == md_vicon['angles_names']
n_frames_vicon = md_vicon['n_frames'] # store number of frames in a separate variable
# generate time vector based on the length and fps of the data
t_vicon = np.linspace(0, len(ang_vicon)*(1/fps_vicon), len(ang_vicon))

# Angle comparison

Create plotting interface to compare the angles acquired from openpose and the Vicon system

In [29]:
# create dropdown object with the angles names
joint_angle = wg.Dropdown(options=ang_names, # set as angles names list
                        description='Angle:', # name description
                        value='Shoulder <- Hip -> Knee', # start with the hip angle
                        disabled=False)

# function to plot the comparison
def plot_2d_comparison(joint_angle):
    # get desired angle index
    angle_idx = ang_names.index(joint_angle)
    plt.close('all') # close previously opened plots
    plt.figure() # open figure
    # plot vicon data for the selected angle
    plt.plot(t_vicon, ang_vicon[:, angle_idx], label='Vicon')
    # plot openpose data for the selected angle
    plt.plot(t_op, ang_op[:, angle_idx],  label='Openpose')
    # separate main joint name for plot title
    angle_name = joint_angle.split('->')[0].split('<-')[-1].replace(" ", "")
    # set plot title
    plt.title("2D Comparation for the {}".format(angle_name))
    plt.xlabel("Time (s)") # set plot x axis label
    plt.ylabel("Angle (Degrees)") # set plot y axis label
    plt.grid(True) # set grid
    plt.legend() # set legend
    plt.show() # show plot

# set interactive plot to refresh plot after a change in the dropdown names
out = wg.interactive_output(plot_2d_comparison, {'joint_angle': joint_angle})
# vertical box to align the angles dropdown and the interactive plot
vbox = wg.VBox([joint_angle, out])
# display vertical box
display(vbox)